In [1]:
from torchvision import datasets, transforms
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
transform=transforms.Compose([transforms.ToTensor()])

In [3]:
MNIST = datasets.MNIST('D:/', train= True, download = False, transform = transform)

In [4]:
train_loader = DataLoader(MNIST, batch_size=32, shuffle=True)

In [5]:
class MNIST_NET(nn.Module):
    def __init__(self):
        super(MNIST_NET, self).__init__()
        self.conv1 = nn.Conv2d(1,20,5)
        self.conv2 = nn.Conv2d(20,50,5)
        self.linear1 = nn.Linear(800,500)
        self.linear2 = nn.Linear(500,10)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,(2,2))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,(2,2))
        x = x.view(-1,4*4*50)
        x =F.relu(self.linear1(x))
        x = self.linear2(x)
        return F.softmax(x, dim = 1)
        

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else'cpu')
model = MNIST_NET().to(device)
#model.load_state_dict(torch.load('Saved_Weights/CNN_11.pt'))

In [8]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [9]:
start_epoch = 0
end_epoch = 10
for epoch in range(start_epoch, end_epoch):
    total_loss = 0
    for idx , (image,label) in enumerate(train_loader):
        model.zero_grad()
        output = model(image.to(device))
        #print (image.size)
        loss = criterion(output, label.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('Epoch {}/{} , Loss: {}'.format(epoch + 1, end_epoch , total_loss/len(train_loader)))
    torch.save(model.state_dict(), 'Saved_Weights/CNN_{}.pt'.format(epoch))

Epoch 1/10 , Loss: 1.525862047513326
Epoch 2/10 , Loss: 1.4849230217615763
Epoch 3/10 , Loss: 1.479650120608012
Epoch 4/10 , Loss: 1.4763073597590128
Epoch 5/10 , Loss: 1.4767597573598226
Epoch 6/10 , Loss: 1.4750546988805135
Epoch 7/10 , Loss: 1.4742090458552042
Epoch 8/10 , Loss: 1.4752289520899455
Epoch 9/10 , Loss: 1.4745552492141725
Epoch 10/10 , Loss: 1.4727225736618041


In [10]:
test_dataset = datasets.MNIST('D:/',train= False, download = False, transform = transform)

In [11]:
test_loader = DataLoader (test_dataset , batch_size= 1)

In [13]:
accuracy = 0
total_loss = 0
with torch.no_grad():
    for idx, (image , labels) in enumerate (test_loader):
        inp = image.to(device).float()
        output = model(inp)
        #plt.imshow(inp.cpu().squeeze(0).squeeze(0))
        #plt.show()
        pred = output.argmax(dim= 1 , keepdim = True)
        #print(pred.item())
        total_loss += criterion(output, labels.to(device)).item()
        if pred == labels.item():
            accuracy +=1
        #print(".................................................................")
    print('Accuracy: ',accuracy/len(test_loader)* 100 , 'Loss: ',total_loss/len(test_loader))


Accuracy:  98.35000000000001 Loss:  1.4777639600157737
